hello world

In [49]:
import pandas as pd
import json
import re
from transition_amr_parser.parse import AMRParser
from transition_amr_parser.amr import AMR

In [ ]:
# read data set

In [5]:
df = pd.read_csv('../dataset/dataset_multilingual.csv') 

In [51]:
languages = set(df['language'])
languages

{'EN'}

In [60]:
sentences = df['en']

In [52]:
for i in range(10):
    print(sentences[i])


Madam President, on a point of order. You will be aware from the press and television that there have been a number of bomb explosions and killings in Sri Lanka. One of the people assassinated very recently in Sri Lanka was Mr Kumar Ponnambalam, who had visited the European Parliament just a few months ago. Would it be appropriate for you, Madam President, to write a letter to the Sri Lankan President expressing Parliament's regret at his and the other violent deaths in Sri Lanka and urging her to do everything she possibly can to seek a peaceful reconciliation to a very difficult situation?



Madam President, on a point of order. I would like your advice about Rule 143 concerning inadmissibility. My question relates to something that will come up on Thursday and which I will then raise again.

The Cunha report on multiannual guidance programmes comes before Parliament on Thursday and contains a proposal in paragraph 6 that a form of quota penalties should be introduced for countries

In [59]:
def splits(text):
    sentences = re.split(r'([.!?])', text)
    sentences = [sentences[i] + sentences[i+1] for i in range(0, len(sentences)-1, 2)]
    return sentences

In [61]:
sentences = list(map(splits, sentences))

In [62]:
print(sentences[0])

['\nMadam President, on a point of order.', ' You will be aware from the press and television that there have been a number of bomb explosions and killings in Sri Lanka.', ' One of the people assassinated very recently in Sri Lanka was Mr Kumar Ponnambalam, who had visited the European Parliament just a few months ago.', " Would it be appropriate for you, Madam President, to write a letter to the Sri Lankan President expressing Parliament's regret at his and the other violent deaths in Sri Lanka and urging her to do everything she possibly can to seek a peaceful reconciliation to a very difficult situation?"]


In [8]:
parser = AMRParser.from_pretrained('AMR3-structbart-L')


load from cache /Users/cricoche/.cache/torch/DATA/AMR3.0/models/amr3.0-structured-bart-large-neur-al/seed42/checkpoint_wiki.smatch_top5-avg.pt
| [en] dictionary: 46088 types
| [actions_nopos] dictionary: 16544 types
----------loading pretrained bart.large model ----------


Using cache found in /Users/cricoche/.cache/torch/hub/pytorch_fairseq_main


---------- task bart rewind: loading pretrained bart.large model ----------


Using cache found in /Users/cricoche/.cache/torch/hub/pytorch_fairseq_main


using CPU for models
pretrained_embed:  bart.large


Using cache found in /Users/cricoche/.cache/torch/hub/pytorch_fairseq_main


Using bart.large extraction in cpu (slow, wont OOM)
Finished loading models
self.machine_config:  /Users/cricoche/.cache/torch/DATA/AMR3.0/models/amr3.0-structured-bart-large-neur-al/seed42/machine_config.json


In [165]:
output_data = []
for i, text in enumerate(sentences):
    penmans = []
    branches = []
    for sentence in text:
        # Tokenize the sentence
        tokens, positions = parser.tokenize(sentence)
        
        # Parse the sentence
        annotations, machines = parser.parse_sentence(tokens)

        # Get AMR graph object
        amr = machines.get_amr()
        
        penman_notation = amr.to_penman(jamr=False, isi=True)
        br = len(amr.edges)
        
        penmans.append(penman_notation)
        branches.append(br)

    print (f"Sentence {i}/{len(sentences)}")

    output_data.append({
            'penman_notation': penmans,
            'branches' : branches
        
        })

print("AMR processing completed.")


In [64]:
penmans = [x['penman_notation'] for x in output_data]
branches = [x['branches'] for x in output_data]

In [65]:
len(penmans)

588

In [70]:
output_df = pd.DataFrame(output_data)
output_df.to_csv('../dataset/amr_output.csv', index=False)



In [73]:
set(df['state'])

{'Austria',
 'Belgium',
 'Bulgaria',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'England',
 'Estonia',
 'Finland',
 'France',
 'Germany',
 'Greece',
 'Hungary',
 'Ireland',
 'Italy',
 'Latvia',
 'Lithuania',
 'Luxembourg',
 'Malta',
 'Netherlands',
 'Northern Ireland',
 'Poland',
 'Portugal',
 'Romania',
 'Scotland',
 'Slovakia',
 'Slovenia',
 'Spain',
 'Sweden',
 'Unknown',
 'Wales'}

In [81]:
states = ['England','Ireland','Northern Ireland','Wales']
filtered_df = df[df['state'].isin(states)]

In [87]:
for s in states:
    print(s + ": " + str(len(df[df['state']==s])))

England: 6053
Ireland: 2810
Northern Ireland: 459
Wales: 193


In [91]:
eng_df = df[df['state']=='England'].sample(frac=1)
irs_df = df[df['state']=='Ireland'].sample(frac=1)
nir_df = df[df['state']=='Northern Ireland'].sample(frac=1)
wal_df = df[df['state']=='Wales'].sample(frac=1)

In [117]:
eng_df.head(3)

,country,euroid,language,name,normalized_name,seq_speaker_id,session_id,type,region,state,missing,reason,da,de,en,es,fr,it,nl
235,United Kingdom,4554,EN,Davies,Davies,57,ep-00-03-14,speech,Lancashire,England,False,NaN,"\nHr. formand, én af glæderne ved at tilbringe...","\nHerr Präsident, eines der Vergnügen von Belg...","\nMr President, one of the joys of spending ti...","\nSeñor Presidente, uno de los placeres que no...","\nMonsieur le Président, l'un des plaisirs que...","\nSignor Presidente, uno dei piaceri che si gu...","\nMijnheer de Voorzitter, één van de geneugten..."
5337,United Kingdom,1428,EN,Balfe (PPE-DE ).,Balfe PPEDE,131,ep-04-04-21,speech,Suffolk,England,False,NaN,"\n\n - Hr. formand, kan De give Parlamentet ...",\n\n – Ich lasse das Parlament darüber entsc...,"\n\n Mr President, could you give the House...","\n\n – Señor Presidente, ¿podría indicar a l...","\n\n - Monsieur le Président, pourriez-vous ...","\n\n – Signor Presidente, potrebbe dare qual...","\n\n - Mijnheer de Voorzitter, kunt u ons ie..."
4155,United Kingdom,4524,EN,Lucas (Verts/ALE),Lucas VertsALE,144,ep-03-06-03,speech,Worcestershire,England,False,NaN,\nEmnet for denne betænkning er yderst vigtigt...,\n. (EN) Der vorliegende Bericht befasst sich ...,\n. The subject of this report is an extremely...,\n. (EN) El tema de este informe es sumamente ...,\nLe sujet de ce rapport est de la plus haute ...,\nL'argomento di questa relazione è estremamen...,\n. (EN) Dit verslag gaat over een buitengewoo...


In [93]:
num_ex_per_class = 193

In [97]:
eng_df = eng_df.head(num_ex_per_class)
irs_df = irs_df.head(num_ex_per_class)
nir_df = nir_df.head(num_ex_per_class)
wal_df = wal_df.head(num_ex_per_class)

In [133]:
def choose_first_texts(texts):
    if(texts[0] == "\n."):
        texts = texts[1:]
    l = len(texts[0])
    texts_list = [texts[0]]
    i = 1
    while i < len(texts) and l + len(texts[i]) <= 500:
        texts_list.append(texts[i])
        l+= len(texts[i])
        i+=1
    return texts_list

In [134]:
def extract_sentence_samples(row):
    return choose_first_texts(splits(row['en']))

In [138]:
def concat(lists):
    l = []
    for lis in lists:
        l+=lis
    return l

In [143]:
eng_sentences = eng_df.apply(extract_sentence_samples, axis=1)
eng_sentences = concat(eng_sentences)

In [145]:
irs_sentences = concat(irs_df.apply(extract_sentence_samples, axis=1))
nir_sentences = concat(nir_df.apply(extract_sentence_samples, axis=1))
wal_sentences = concat(wal_df.apply(extract_sentence_samples, axis=1))

In [148]:
sentences = [eng_sentences,irs_sentences,nir_sentences,wal_sentences]
for s in sentences:
    print(str(len(s)))

638
666
635
603


In [169]:
label = len(eng_sentences) * ['eng'] + len(irs_sentences) * ['irs'] + len(nir_sentences) * ['nir'] + len(wal_sentences) * ['wal']
sentence = concat(sentences)

In [171]:
sentence = [s.strip('\n\xa0') for s in sentence]

In [161]:
class_df = pd.DataFrame({'sentence' : sentence, 'label' : label}).sample(frac=1)

In [162]:
print(class_df.head(5))

                                               sentence label
762    Like previous speakers, I welcome it because ...   irs
1708                    This is a very worrying aspect.   nir
1796  Mr President, recently, along with other passe...   nir
1162  Mr President, I am speaking basically on the A...   irs
546    Mr President, the Commissioner is of course n...   eng


In [163]:
class_df.to_csv('../dataset/classify_df.csv', index=False)

In [166]:
def sentences_to_amr(sentences):
    output_data = []
    for i, sentence in enumerate(sentences):
        penmans = []
        branches = []
        tokens, positions = parser.tokenize(sentence)
        annotations, machines = parser.parse_sentence(tokens)
        amr = machines.get_amr()
            
        penman_notation = amr.to_penman(jamr=False, isi=True)
        br = len(amr.edges)
    
        print (f"Sentence {i}/{len(sentences)}")
    
        output_data.append({
                'penman_notation': penman_notation,
                'branches' : br
            })
    
    print("AMR processing completed.")
    return output_data


In [172]:
sentence

["Mr President, one of the joys of spending time in Belgium and France is the pleasure of these countries' chocolate.",
 ' It is wonderful, but millions of people in Europe also enjoy the British product.',
 ' None of them come to any harm in the process.',
 ' Whatever its content they are happy to call this product chocolate.',
 'It is insulting for us to play semantic games about the definition of this word.',
 ' The citizens we represent are not fools.',
 '  Mr President, could you give the House some indication as to whether you intend to break for lunch?',
 ' There are a lot of votes and there are a couple of official delegations, one from the Australian Parliament, waiting for our colleague Mr Sturdy to host lunch with them.',
 ' It is also 2.',
 '10 p.',
 'm.',
 '!',
 ' The subject of this report is an extremely important one.',
 ' That is why the Green/EFA Group was one of the instigators of an ITRE own-initiative report on the subject of regional integration, particularly with

In [173]:
sentence_amrs = sentences_to_amr(sentence)

Running on batch size: 1
1


decoding: 100%|█████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.88s/it]


Sentence 0/2542
Running on batch size: 1
1


decoding: 100%|█████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.13s/it]


Sentence 1/2542
Running on batch size: 1
1


...

Sentence 2539/2542
Running on batch size: 1
1


decoding: 100%|█████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.34s/it]


Sentence 2540/2542
Running on batch size: 1
1


decoding: 100%|█████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.49s/it]

Sentence 2541/2542
AMR processing completed.


In [174]:
class_amr_df = pd.DataFrame({
    'sentence': sentence,
    'label' : label,
    'penman' : [x['penman_notation'] for x in sentence_amrs],
    'branches': [x['branches'] for x in sentence_amrs],
})

In [175]:
class_amr_df.to_csv('../dataset/classify_amr_df.csv', index=False)